In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering_new.ex2 import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

In [ ]:
data_titanic_path = '../input/tabular-playground-series-apr-2021/train.csv'
# my_data = pd.read_csv(data_titanic_path, index_col='PassengerId')
my_data = pd.read_csv(data_titanic_path)
my_data.head()

In [ ]:
my_data = my_data.drop('Name', axis=1)
my_data.head()

In [ ]:
data_titanic_path_test = '../input/tabular-playground-series-apr-2021/test.csv'
# my_data = pd.read_csv(data_titanic_path, index_col='PassengerId')
my_test_data = pd.read_csv(data_titanic_path_test)
my_test_data.head()

In [ ]:
my_test_data = my_test_data.drop('Name', axis=1)
my_test_data.head()

In [ ]:
my_data.dtypes

In [ ]:
my_test_data.isnull().values.any()

In [ ]:
categorical_cols = my_data[my_data.columns].select_dtypes(exclude=['int64','float64']).columns

In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: my_data[col].nunique(), categorical_cols))
d = dict(zip(categorical_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
from sklearn.preprocessing import LabelEncoder
import plotly.figure_factory as ff

primary_bgcolor = "#f4f0ea"

le = LabelEncoder()
le_data = my_data.copy().drop(columns=['PassengerId'])

pure_cat_cols = my_data[my_data.columns].select_dtypes(exclude=['int64','float64']).columns
for col in pure_cat_cols:
    le.fit(le_data[col])
    le_data[col] = le.transform(le_data[col])
    #my_test_data[col] = le.transform(my_test_data[col])

corrdata = le_data

## correlation 
corr = corrdata.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr1 = corr.mask(mask)

fig = ff.create_annotated_heatmap(
    z=corr1.to_numpy().round(2),
    x=list(corr1.index.values),
    y=list(corr1.columns.values),       
    xgap=3, ygap=3,
    zmin=0, zmax=1,
    colorscale='blugrn',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)

fig.update_layout(
    title_text='<span style="font-size:32px; font-family:Times New Roman">Features Correlation Matrix</span>', 
    font_family="Serif",
    titlefont={'size': 24},
    width=800, height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed', 
    paper_bgcolor=primary_bgcolor,
    plot_bgcolor=primary_bgcolor,
    margin=dict(l=70, r=70, t=70, b=70, pad=1),
)

fig.show()

In [ ]:
le_data.head()

In [ ]:
important_features = ["Sex", "Pclass", "Embarked", "Fare", "Cabin", "Survived"]

In [ ]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in categorical_cols if my_data[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(categorical_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

In [ ]:
le = LabelEncoder()
high_cardinality_cols_to_drop = ['Cabin', 'Ticket']
my_data = my_data.drop(high_cardinality_cols_to_drop, axis=1)
my_test_data = my_test_data.drop(high_cardinality_cols_to_drop, axis=1)
le_data = my_data.copy().drop(columns=['PassengerId'])

for col in low_cardinality_cols:
    le.fit(le_data[col])
    le_data[col] = le.transform(le_data[col])
    #my_test_data[col] = le.transform(my_test_data[col])

In [ ]:
le_data.head()

In [ ]:
my_test_data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import plotly.figure_factory as ff

primary_bgcolor = "#f4f0ea"

corrdata = le_data

## correlation 
corr = corrdata.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr1 = corr.mask(mask)

fig = ff.create_annotated_heatmap(
    z=corr1.to_numpy().round(2),
    x=list(corr1.index.values),
    y=list(corr1.columns.values),       
    xgap=3, ygap=3,
    zmin=0, zmax=1,
    colorscale='blugrn',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)

fig.update_layout(
    title_text='<span style="font-size:32px; font-family:Times New Roman">Features Correlation Matrix</span>', 
    font_family="Serif",
    titlefont={'size': 24},
    width=800, height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed', 
    paper_bgcolor=primary_bgcolor,
    plot_bgcolor=primary_bgcolor,
    margin=dict(l=70, r=70, t=70, b=70, pad=1),
)

fig.show()

In [ ]:
# Get Correlation
print(le_data[le_data.columns].corrwith(le_data.Survived))

In [ ]:
new_import_features = ['Sex', 'Embarked', 'Pclass', 'Fare']

In [ ]:
le_data[new_import_features].isnull().values.any()

In [ ]:
le_data_important_features = le_data[new_import_features]

dict_null = {}
for col in new_import_features:
    dict_null[col] = le_data_important_features[col].isnull().sum()
dict_null

In [ ]:
my_data.head()

In [ ]:
new_my_data = my_data[['Pclass', 'Sex', 'Fare', 'Embarked']]
new_my_test = my_test_data[['Pclass', 'Sex', 'Fare', 'Embarked']]
y = my_data['Survived']

In [ ]:
new_my_data = new_my_data.fillna(method='bfill', axis=0)
new_my_test = new_my_test.fillna(method='bfill', axis=0)

In [ ]:
new_categorical_cols = new_my_data[new_my_data.columns].select_dtypes(exclude=['int64','float64']).columns
new_categorical_cols

In [ ]:
new_my_test.head()

In [ ]:
new_my_data.head()

In [ ]:
new_my_test[new_import_features].isnull().values.any()

In [ ]:
new_my_data.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(new_my_data[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(new_my_test[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = new_my_data.index
OH_cols_valid.index = new_my_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = new_my_data.drop(new_categorical_cols, axis=1)
num_X_valid = new_my_test.drop(new_categorical_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [ ]:
num_X_train.head()

In [ ]:
OH_cols_train.head()

In [ ]:
#le_data_without_null = le_data[new_import_features].dropna()

In [ ]:
#le_data_without_null.describe()

In [ ]:
#my_test_data = my_test_data[new_import_features]

#dict_null = {}
#for col in new_import_features:
#    dict_null[col] = my_test_data[col].isnull().sum()
#dict_null

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
X = le_data.drop('Survived', axis=1).fillna(method='bfill', axis=0).copy()

mi_scores = make_mi_scores(X, y)
mi_scores

In [ ]:
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores.head(20))
# plot_mi_scores(mi_scores.tail(20))  # uncomment to see bottom 20

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [ ]:
print(len(OH_X_train))
print(len(y))

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, OH_X_train, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

In [ ]:
model = RandomForestRegressor(n_estimators=50, random_state=0)

In [ ]:
my_data.head()

In [ ]:
OH_X_train.head()

In [ ]:
len(OH_X_valid)

In [ ]:
my_test_data.head()

In [ ]:
model.fit(OH_X_train, y)

# Fill in the line below: get test predictions
preds_test = model.predict(OH_X_valid)

# Save test predictions to file
output = pd.DataFrame({'PassengerId': my_test_data.PassengerId,
                       'Survived': preds_test})
output.to_csv('submission.csv', index=False)